In [1]:
pip install huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import login
login(token="")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
from huggingface_hub import hf_hub_download

hf_hub_download(
    repo_id="facebook/UMA",
    filename="checkpoints/uma-s-1p1.pt",
    token="", 
    local_dir="."
)


checkpoints/uma-s-1p1.pt:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

'checkpoints/uma-s-1p1.pt'

optimizing multi-input POSACR

In [4]:
import os
from ase.io import read, write
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.optimize import LBFGS

# set the path
input_dir = "/home/users/yeong/work/25_s/VASPsol++/03.oc20/00-211/input_H+"  #directory
output_dir = "/home/users/yeong/work/25_s/VASPsol++/03.oc20/00-211/output_H+"
os.makedirs(output_dir, exist_ok=True)

# setting calculator
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")

# list for saving the results
results = []

# for loop
for fname in os.listdir(input_dir):
    if not fname.startswith("POSCAR"):
        continue
    in_path = os.path.join(input_dir, fname)
    atoms = read(in_path, format="vasp")

    # optimizing
    calc = FAIRChemCalculator(predictor, task_name="oc20")
    atoms.calc = calc

    opt = LBFGS(atoms, logfile=None)
    opt.run(fmax=0.05, steps=100)

    energy = atoms.get_potential_energy()
    out_path = os.path.join(output_dir, f"{fname}_opt.vasp")
    write(out_path, atoms, format="vasp")

    results.append({
        "file": out_path,
        "energy": energy,
        "name": fname
    })
    print(f"[{fname}] E = {energy:.4f} eV")

# select the best structure
best = sorted(results, key=lambda x: x["energy"])[0]
best_path = os.path.join(output_dir, "best_POSCAR")
write(best_path, read(best["file"], format="vasp"), format="vasp")

print(f"\ best structure : {best['name']}, E = {best['energy']:.4f} eV")
print(f"path: {best_path}")


/home/users/yeong/.conda/envs/uma/lib/python3.10/site-packages/fairchem/core/models/uma/escn_md.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Jd_list = torch.load(os

[POSCAR_3] E = -188.9880 eV
[POSCAR_9] E = -189.9334 eV
[POSCAR_25] E = -189.8627 eV
[POSCAR_17] E = -189.0277 eV
[POSCAR_31] E = -189.2697 eV
[POSCAR_10] E = -189.8946 eV
[POSCAR_22] E = -190.0162 eV
[POSCAR_28] E = -190.0158 eV
[POSCAR_4] E = -188.9863 eV
[POSCAR_26] E = -189.3171 eV
[POSCAR_0] E = -190.1228 eV
[POSCAR_14] E = -190.2380 eV
[POSCAR_19] E = -189.0302 eV
[POSCAR_13] E = -189.8576 eV
[POSCAR_7] E = -189.9574 eV
[POSCAR_21] E = -189.9336 eV
[POSCAR_11] E = -189.1073 eV
[POSCAR_29] E = -189.2125 eV
[POSCAR_23] E = -190.0200 eV
[POSCAR_5] E = -189.0150 eV
[POSCAR_30] E = -189.2686 eV
[POSCAR_8] E = -190.2402 eV
[POSCAR_2] E = -190.0197 eV
[POSCAR_24] E = -189.8890 eV
[POSCAR_16] E = -189.2084 eV
[POSCAR_12] E = -189.1746 eV
[POSCAR_18] E = -189.0021 eV
[POSCAR_6] E = -190.2367 eV
[POSCAR_20] E = -190.2379 eV
[POSCAR_27] E = -189.3182 eV
[POSCAR_1] E = -190.0401 eV
[POSCAR_15] E = -189.1763 eV
\ best structure : POSCAR_8, E = -190.2402 eV
path: /home/users/yeong/work/25_s/VA